In [ ]:
import os
import copy
from coffea import hist
from coffea import util
import numpy as np
import itertools
import pandas as pd

In [ ]:
directory = 'CoffeaOutputsForTriggerAnalysis/SingleMu/LooseBTag/2016/'

In [ ]:
Data_num2016 = util.load(directory+'TTbarRes_0l_SingleMu2016_Data_TriggerAnalysis.coffea')

In [ ]:
print("-------Unweighted SingleMu2016_Data--------")
for i,j in Data_num2016['cutflow'].items():        
    print( '%20s : %12d' % (i,j) )

In [ ]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [ ]:
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)

In [ ]:
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

# ---- Reiterate categories ---- #
ttagcats = ["0t", ">=1t"]
# btagcats = ["0b", "1b", "2b"]
# ycats = ['cen', 'fwd']

# list_of_cats = [ t+b+y for t,b,y in itertools.product( ttagcats, btagcats, ycats) ]
# list_of_bcats = [ b+y for b,y in itertools.product( btagcats, ycats) ]

# # ---- List the Histograms Here ---- #
# list_of_hists = ('ttbarmass', 'jetpt', 'jeteta', 'jetphi', 'jety', 'jetdy', 'probept', 'probep')

In [ ]:
maindirectory = os.getcwd() 

# Plots without uncertainty bars included

In [ ]:
SaveDirectory = maindirectory + '/TriggerAnalysisPlots/LooseBTag/2016/' # Directory where trigger analysis plots will be saved
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already

In [ ]:
for cat in ttagcats:
    #---- Histogram Window Config. ----# 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, ax = plt.subplots(
        nrows=1,
        ncols=1,
        figsize=(7,7),
    )
#     fig.subplots_adjust(hspace=.07)
    ax.set_title('Trigger Efficiency')
    
    Numerator1 = Data_num2016['condition1_numerator'].integrate('anacat', cat).integrate('Jet_sdMass').integrate('dataset', 'SingleMu2016_Data')
    Numerator2 = Data_num2016['condition2_numerator'].integrate('anacat', cat).integrate('Jet_sdMass').integrate('dataset', 'SingleMu2016_Data')
    Numerator3 = Data_num2016['condition3_numerator'].integrate('anacat', cat).integrate('Jet_sdMass').integrate('dataset', 'SingleMu2016_Data')
    Numerator4 = Data_num2016['condition4_numerator'].integrate('anacat', cat).integrate('Jet_sdMass').integrate('dataset', 'SingleMu2016_Data')
    Denominator = Data_num2016['condition_denominator'].integrate('anacat', cat).integrate('Jet_sdMass').integrate('dataset', 'SingleMu2016_Data')
    
    # print(Denominator.values())
    
    # TestPlot = hist.plot1d(Denominator, ax=ax, clear=False, error_opts={'marker': '.', 'markersize': 7., 'color': 'k', 'elinewidth': 1})
    
    EffPlot1 = hist.plotratio(num = Numerator1, denom = Denominator, ax = ax, clear=False,
                   error_opts={'marker': '.', 'markersize': 7., 'color': 'k', 'elinewidth': 0})
                   #unc = 'num')
    EffPlot2 = hist.plotratio(num = Numerator2, denom = Denominator, ax = ax, clear=False,
                   error_opts={'marker': 'x', 'markersize': 7., 'color': 'g', 'elinewidth': 0})
                   #unc = 'num')
    EffPlot3 = hist.plotratio(num = Numerator3, denom = Denominator, ax = ax, clear=False,
                   error_opts={'marker': 's', 'markersize': 6., 'color': 'r', 'elinewidth': 0})
                   #unc = 'num')
    EffPlot4 = hist.plotratio(num = Numerator4, denom = Denominator, ax = ax, clear=False,
                   error_opts={'marker': '*', 'markersize': 7., 'color': 'b', 'elinewidth': 0})
                   #unc = 'num')
    
    ax.set_ylim(bottom=0.0, top=1.1)
    ax.set_xlim(220, 2200)
    ax.set_ylabel('Efficiency')
    leg = ax.legend(labels=[r'$Condition_1$',r'$Condition_2$',r'$Condition_3$',r'$Condition_4$'], bbox_to_anchor=(1.35, 1.), loc='upper right')
    
    Cat_legend_entry = None
    if cat == '0t':
        Cat_legend_entry = "No $m_{SD}$ cut"
        fnc = '_noSD'
    else:
        Cat_legend_entry = "$105GeV < m_{SD} < 210GeV$"
        fnc = '_SD'
    leg2 = plt.text(1.35, 1.07, Cat_legend_entry,
                fontsize=16,
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes
               )
    filename = 'TriggerEffs_NOunc' + fnc +'.png'
    filename_zoom = 'TriggerEffs_NOunc_zoom' + fnc + '.png'
    # plt.savefig(SaveDirectory+filename_zoom, bbox_inches="tight")
    # print(SaveDirectory+filename_zoom)

# Plots with uncertainty bars included

In [ ]:
for cat in ttagcats:
    #---- Histogram Window Config. ----# 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, ax = plt.subplots(
        nrows=1,
        ncols=1,
        figsize=(7,7),
    )
#     fig.subplots_adjust(hspace=.07)
    ax.set_title('Trigger Efficiency; ' + cat)
    
    Numerator1 = Data_num2016['condition1_numerator'].integrate('anacat', cat).integrate('Jet_sdMass').integrate('dataset', 'SingleMu2016_Data')
    Numerator2 = Data_num2016['condition2_numerator'].integrate('anacat', cat).integrate('Jet_sdMass').integrate('dataset', 'SingleMu2016_Data')
    Numerator3 = Data_num2016['condition3_numerator'].integrate('anacat', cat).integrate('Jet_sdMass').integrate('dataset', 'SingleMu2016_Data')
    Numerator4 = Data_num2016['condition4_numerator'].integrate('anacat', cat).integrate('Jet_sdMass').integrate('dataset', 'SingleMu2016_Data')
    Denominator = Data_num2016['condition_denominator'].integrate('anacat', cat).integrate('Jet_sdMass').integrate('dataset', 'SingleMu2016_Data')
    
    # print(Denominator.values())
    
    # TestPlot = hist.plot1d(Denominator, ax=ax, clear=False, error_opts={'marker': '.', 'markersize': 7., 'color': 'k', 'elinewidth': 1})
    
    EffPlot1 = hist.plotratio(num = Numerator1, denom = Denominator, ax = ax, clear=False,
                   error_opts={'marker': '.', 'markersize': 7., 'color': 'k', 'elinewidth': 1},
                   unc = 'num')
    EffPlot2 = hist.plotratio(num = Numerator2, denom = Denominator, ax = ax, clear=False,
                   error_opts={'marker': 'x', 'markersize': 7., 'color': 'g', 'elinewidth': 1},
                   unc = 'num')
    EffPlot3 = hist.plotratio(num = Numerator3, denom = Denominator, ax = ax, clear=False,
                   error_opts={'marker': 's', 'markersize': 6., 'color': 'r', 'elinewidth': 1},
                   unc = 'num')
    EffPlot4 = hist.plotratio(num = Numerator4, denom = Denominator, ax = ax, clear=False,
                   error_opts={'marker': '*', 'markersize': 7., 'color': 'b', 'elinewidth': 1},
                   unc = 'num')
    
    ax.set_ylim(bottom=0.0, top=1.1)
    ax.set_xlim(220, 2200)
    ax.set_ylabel('Efficiency')
    leg = ax.legend(labels=[r'$Condition_1$',r'$Condition_2$',r'$Condition_3$',r'$Condition_4$'], bbox_to_anchor=(1.35, 1.), loc='upper right')
    
    Cat_legend_entry = None
    if cat == '0t':
        Cat_legend_entry = "No $m_{SD}$ cut"
        fnc = '_noSD'
    else:
        Cat_legend_entry = "$105GeV < m_{SD} < 210GeV$"
        fnc = '_SD'
    leg2 = plt.text(1.35, 1.07, Cat_legend_entry,
                fontsize=16,
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes
               )
    filename = 'TriggerEffs_unc' + fnc +'.png'
    filename_zoom = 'TriggerEffs_unc_zoom' + fnc + '.png'
    # plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    # print(SaveDirectory+filename)
    

In [ ]:
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")
import mplhep as hep
from hist.intervals import ratio_uncertainty

In [ ]:
def plotratio2d(numerator, denominator, ax=None, cmap='Blues', cbar=True, norm_min=0., norm_max=1., alpha=None):
    NumeratorAxes = numerator.axes()
    DenominatorAxes = denominator.axes()
    
    # integer number of bins in this axis #
    NumeratorAxis1_BinNumber = NumeratorAxes[0].size - 3 # Subtract 3 to remove overflow
    NumeratorAxis2_BinNumber = NumeratorAxes[1].size - 3
    
    DenominatorAxis1_BinNumber = DenominatorAxes[0].size - 3 
    DenominatorAxis2_BinNumber = DenominatorAxes[1].size - 3 
    
    if(NumeratorAxis1_BinNumber != DenominatorAxis1_BinNumber 
       or NumeratorAxis2_BinNumber != DenominatorAxis2_BinNumber):
        raise Exception('Numerator and Denominator axes are different sizes; Cannot perform division.')
    else:
        Numerator = numerator.to_hist()
        Denominator = denominator.to_hist()

        ratio = Numerator / Denominator.values()
        
        return hep.hist2dplot(ratio, ax=ax, cmap=cmap, norm=colors.Normalize(norm_min,norm_max), cbar=cbar, alpha=alpha)

In [ ]:
for cat in ttagcats:
    #---- Histogram Window Config. ----# 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(
        nrows=1,
        ncols=4,
        figsize=(17,5),
    )
#     fig.subplots_adjust(hspace=.07)
    # plt.set_title('Trigger Efficiency; ' + cat)
    
    Numerator1 = Data_num2016['condition1_numerator'].integrate('anacat', cat).integrate('dataset', 'SingleMu2016_Data')
    Numerator2 = Data_num2016['condition2_numerator'].integrate('anacat', cat).integrate('dataset', 'SingleMu2016_Data')
    Numerator3 = Data_num2016['condition3_numerator'].integrate('anacat', cat).integrate('dataset', 'SingleMu2016_Data')
    Numerator4 = Data_num2016['condition4_numerator'].integrate('anacat', cat).integrate('dataset', 'SingleMu2016_Data')
    Denominator = Data_num2016['condition_denominator'].integrate('anacat', cat).integrate('dataset', 'SingleMu2016_Data')
    
    # print(Denominator.values())
    
    # TestPlot = hist.plot1d(Denominator, ax=ax, clear=False, error_opts={'marker': '.', 'markersize': 7., 'color': 'k', 'elinewidth': 1})
    
    EffPlot1 = plotratio2d(Numerator1, Denominator, ax=ax1, cmap='Blues')
        
    EffPlot2 = plotratio2d(Numerator2, Denominator, ax=ax2, cmap='Blues')
    
    EffPlot3 = plotratio2d(Numerator3, Denominator, ax=ax3, cmap='Blues')
    
    EffPlot4 = plotratio2d(Numerator4, Denominator, ax=ax4, cmap='Blues')
    
    # ax.set_ylim(bottom=0.0, top=1.1)
    # ax.set_xlim(220, 2200)
    # ax.set_ylabel('Efficiency')
    # leg = ax.legend(labels=[r'$Condition_1$',r'$Condition_2$',r'$Condition_3$',r'$Condition_4$'], bbox_to_anchor=(1.35, 1.), loc='upper right')
    
    # Cat_legend_entry = None
    # if cat == '0t':
    #     Cat_legend_entry = "No $m_{SD}$ cut"
    #     fnc = '_noSD'
    # else:
    #     Cat_legend_entry = "$105GeV < m_{SD} < 210GeV$"
    #     fnc = '_SD'
    # leg2 = plt.text(1.35, 1.07, Cat_legend_entry,
    #             fontsize=16,
    #             horizontalalignment='right',
    #             verticalalignment='top',
    #             transform=ax.transAxes
               # )
    filename = 'TriggerEffs_unc' + fnc +'.png'
    filename_zoom = 'TriggerEffs_unc_zoom' + fnc + '.png'
    # plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    # print(SaveDirectory+filename)
    

In [ ]:
for cat in ttagcats:
    #---- Histogram Window Config. ----# 
    plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 18,
    'axes.labelsize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12
    })
    fig, ax = plt.subplots(
        # nrows=1,
        # ncols=4,
        figsize=(17,5),
    )
#     fig.subplots_adjust(hspace=.07)
    ax.set_title('Trigger Efficiency; ' + cat)

    Numerator4 = Data_num2016['condition4_numerator'].integrate('anacat', cat).integrate('dataset', 'SingleMu2016_Data')
    Denominator = Data_num2016['condition_denominator'].integrate('anacat', cat).integrate('dataset', 'SingleMu2016_Data')
    
    EffPlot4 = plotratio2d(Numerator4, Denominator, ax=ax, cmap='plasma')
    
    ax.set_xticks([200, 800, 840, 880, 920, 960, 1000, 1200, 1400, 1600, 1800, 2000])
    ax.set_xticklabels(['.20', '.80', '.84', '.88', '.92', '.96', '1.0', '1.2', '1.4', '1.6', '1.8', '2.0'])
    ax.set_xlabel(r'$Jet\ H_T\ [TeV]$')
    # ax.set_ylim(bottom=0.0, top=1.1)
    # ax.set_xlim(220, 2200)
    # ax.set_ylabel('Efficiency')
    # leg = ax.legend(labels=[r'$Condition_1$',r'$Condition_2$',r'$Condition_3$',r'$Condition_4$'], bbox_to_anchor=(1.35, 1.), loc='upper right')
    
    # Cat_legend_entry = None
    # if cat == '0t':
    #     Cat_legend_entry = "No $m_{SD}$ cut"
    #     fnc = '_noSD'
    # else:
    #     Cat_legend_entry = "$105GeV < m_{SD} < 210GeV$"
    #     fnc = '_SD'
    # leg2 = plt.text(1.35, 1.07, Cat_legend_entry,
    #             fontsize=16,
    #             horizontalalignment='right',
    #             verticalalignment='top',
    #             transform=ax.transAxes
               # )
    filename = 'TriggerEffs_unc' + fnc +'.png'
    filename_zoom = 'TriggerEffs_unc_zoom' + fnc + '.png'
    # plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    # print(SaveDirectory+filename)
    